# Imports

In [10]:
# Imports
import numpy as np
import pandas as pd
from tabulate import tabulate
import matplotlib.pyplot as plt

# Some helper functions

In [11]:
def calculate_reactive_pf(power_factor: float) -> float:
    """
    Calculate sin(theta) from power factor.

    Parameters
    ----------
    power_factor: float
        Power factor of the load

    Returns
    -------
    float
        sin(theta) of the load
    """
    return np.sin(np.arccos(power_factor))

In [12]:
s_base = 100e6  # base power in VA

# Transformer Reactances

In [13]:
t_dict: dict[str, list[int | float]] = {
    "from": [3, 5, 7, 9, ],
    "to": [4, 6, 8, 10],
    "rating": [120, 75, 10, 3],
    "r": [0, 0, 0, 0, ],
    "x": [10, 7.5, 5.0, 5.0],
}

header_transformer: list[str] = [f"{"From":^5}", f"{"To":^5}", f"{"Rating(MVA)":^15}", f"{"Z (pu)":^15}"]
table_transformer: list[list[str]] = []

for i in range(len(t_dict["from"])):
    rating = t_dict["rating"][i] * 10 ** 6
    r = t_dict["r"][i]
    x = t_dict["x"][i]
    z = r / 100 + 1j * x / 100
    z_new = z * s_base / rating
    r_new, x_new = z_new.real, z_new.imag
    r_sign = "+" if r_new >= 0 else "-"
    x_sign = "+" if x_new >= 0 else "-"
    table_transformer.append(
        [
            f"{t_dict["from"][i]:^5}",
            f"{t_dict["to"][i]:^5}",
            f"{t_dict["rating"][i]:^15}",
            f"{r_sign:^1} {abs(r_new):^5.5f} {x_sign:^1} {abs(x_new):^5.5f}"
        ]
    )

print(tabulate(table_transformer, headers=header_transformer, tablefmt="pipe", numalign="center", stralign="center"))

|  From   |   To    |    Rating(MVA)    |       Z (pu)        |
|:-------:|:-------:|:-----------------:|:-------------------:|
|    3    |    4    |        120        | + 0.00000 + 0.08333 |
|    5    |    6    |        75         | + 0.00000 + 0.10000 |
|    7    |    8    |        10         | + 0.00000 + 0.50000 |
|    9    |   10    |         3         | + 0.00000 + 1.66667 |


# Line Impedances

In [14]:
tl_dict: dict[str, list[int | float]] = {
    "from": [1, 1, 2, 4, 6, 8, ],
    "to": [2, 3, 3, 5, 7, 9, ],
    "v_base": [345, 345, 345, 132, 33, 11, ],
    "rating": [150, 150, 150, 115, 11, 10, ],
    "length": [100, 150, 50, 20, 20, 5, ],
    "r": [0.119, 0.119, 0.119, 0.068, 0.0805, 0.13, ],
    "x": [0.595, 0.595, 0.595, 0.404, 0.342, 0.0969, ],
}

header_line: list[str] = [f"{"From":^5}", f"{"To":^5}", f"{"Rating(MVA)":^15}", f"{"Z (pu)":^15}"]
table_line: list[list[str]] = []

for i in range(len(tl_dict["from"])):
    x_base = (tl_dict["v_base"][i] * 10 ** 3) ** 2 / s_base
    l = tl_dict["length"][i]
    r = (tl_dict["r"][i] * l) / x_base
    x = (tl_dict["x"][i] * l) / x_base
    z = r + 1j * x
    r_sign = "+" if r >= 0 else "-"
    x_sign = "+" if x >= 0 else "-"
    table_line.append(
        [
            f"{tl_dict["from"][i]:^5}",
            f"{tl_dict["to"][i]:^5}",
            f"{tl_dict["rating"][i]:^15}",
            f"{x_base:^5.5f}",
            f"{r_sign:^1} {abs(r):^5.5f} {x_sign:^1} {abs(x):^5.5f} p.u."
        ]
    )

print(tabulate(table_line, headers=header_transformer, tablefmt="pipe", numalign="center", stralign="center"))

|  From   |   To    |    Rating(MVA)    |         Z (pu)           |
|:-------:|:-------:|:-----------------:|:------------------------:|
|    1    |    2    |        150        | + 0.01000 + 0.04999 p.u. |
|    1    |    3    |        150        | + 0.01500 + 0.07498 p.u. |
|    2    |    3    |        150        | + 0.00500 + 0.02499 p.u. |
|    4    |    5    |        115        | + 0.00781 + 0.04637 p.u. |
|    6    |    7    |        11         | + 0.14784 + 0.62810 p.u. |
|    8    |    9    |        10         | + 0.53719 + 0.40041 p.u. |
